# InSAR time series analysis with HyP3 and MintPy

This notebook shows how to do time-series analysis using HyP3 product with MintPy. It requires `hyp3_sdk` and `MintPy`:

+ run `conda install --yes -c conda-forge hyp3_sdk ipywidgets` to install `hyp3_sdk`
+ check the [installation page](https://github.com/insarlab/MintPy/blob/main/docs/installation.md) to install `MintPy`

## 0. Initial setup of the notebook

The cell below performs the intial setup of the notebook and must be **run every time the notebook (re)starts**. It imports necessary modules and defines the processing location.

In [ ]:
%matplotlib inline
import os
import re
import glob
import zipfile
import numpy as np
import hyp3_sdk as sdk
from pathlib import Path
from osgeo import gdal
from mintpy import view, tsview

proj_dir = os.path.expanduser('~/data/test/HonshuSenDT119')
proj_name = os.path.basename(proj_dir)
hyp3_dir = os.path.join(proj_dir, 'hyp3')
mintpy_dir = os.path.join(proj_dir, 'mintpy')

os.makedirs(hyp3_dir, exist_ok=True)
os.makedirs(mintpy_dir, exist_ok=True)

## 1. Search SLC filenames

Get the SLC filenames to be used for InSAR processing with HyP3. 

One could use the ASF vertex (https://serach.asf.alaska.edu) for it.

In [ ]:
reference_granule = 'S1A_IW_SLC__1SSV_20160112T205113_20160112T205140_009466_00DBA8_C430'

secondary_granules = [
    'S1A_IW_SLC__1SSV_20160205T205112_20160205T205139_009816_00E5D1_997E',
    'S1A_IW_SLC__1SSV_20160229T205112_20160229T205139_010166_00EFF4_357E',
    'S1A_IW_SLC__1SSV_20160324T205112_20160324T205139_010516_00F9E8_FA1C',
    'S1A_IW_SLC__1SSV_20160417T205113_20160417T205140_010866_010443_7429',
    'S1A_IW_SLC__1SSV_20160511T205115_20160511T205142_011216_010F42_BD06',
    'S1A_IW_SLC__1SSV_20160604T205107_20160604T205135_011566_011A9C_EDD4',
    'S1A_IW_SLC__1SSV_20160628T205108_20160628T205136_011916_01259C_EE11'
]

## 2. Run `hyp3_sdk` application

+ to submit jobs for InSAR processing and 
+ to download the hyp3 products

In [ ]:
os.chdir(hyp3_dir)
print('Go to directory:', hyp3_dir)

# initiate HyPe object
hyp3 = sdk.HyP3(prompt = True)
insar_jobs = sdk.Batch()

# submit InSAR jobs
for i, secondary_granule in enumerate(secondary_granules):
    gen_geometry = i == 0
    insar_jobs += hyp3.submit_insar_job(reference_granule,
                                        secondary_granule,
                                        name=projectname,
                                        include_inc_map=gen_geometry,
                                        include_dem=gen_geometry)

# watch InSAR jobs
insar_jobs = hyp3.watch(insar_jobs)

# download and uncompress InSAR results
zip_files = insar_jobs.download_files()

for zip_file in zip_files:
    with zipfile.ZipFile(zip_file) as zip_ref:
        zip_ref.extractall(hyp3_dir)

# prepared data structure are:
! tree {hyp3_dir}

## 3. Prepare InSAR files for MintPy time series analysis

By clipping the GeoTIFF files into the same grids using `gdal`.

In [ ]:
fnames = glob.glob(os.path.join(hyp3_dir, '*/*.tif'))

# determine the smallest area covered by all input files
corners = [gdal.Info(f, format='json')['cornerCoordinates'] for f in fnames]
ulx = max(corner['upperLeft'][0] for corner in corners)
uly = min(corner['upperLeft'][1] for corner in corners)
lrx = min(corner['lowerRight'][0] for corner in corners)
lry = max(corner['lowerRight'][1] for corner in corners)

# subset all input files to these common coordinates
for fname in fnames:
    fname_out = fname.replace('.tif', '_clip.tif')
    gdal.Translate(destName=fname_out, srcDS=fname, projWin=[ulx, uly, lrx, lry])

# show the data structure
! tree {hyp3_dir}

## 4. Run MintPy routine workflow `smallbaselineApp.py`

### 4.1 Prepare the template file

In [ ]:
config_txt = f'''
mintpy.load.processor        = hyp3
##---------interferogram datasets:
mintpy.load.unwFile          = {hyp3_dir}/*/*unw_phase_clip.tif
mintpy.load.corFile          = {hyp3_dir}/*/*corr_clip.tif
##---------geometry datasets:
mintpy.load.demFile          = {hyp3_dir}/*/*dem_clip.tif
mintpy.load.incAngleFile     = {hyp3_dir}/*/*inc_map_clip.tif
'''

# write to file
config_file = os.path.join(mintpy_dir, f'{proj_name}.txt')
with open(config_file, 'w') as fid:
    fid.write(config_txt)
print('write file:', config_file)

### 4.2 Run Time-series Analysis application

In [ ]:
! smallbaselineApp.py {config_file} --work-dir {mintpy_dir}

### 4.3 Display the analysis results

There are a few scripts used to display the analysis results. There are in the MINTPY_HOME/mintpy. Here we show two majoy disaply scripts.

In [ ]:
view.main(['mintpy/velocity.h5'])

In [ ]:
tsview.main(['mintpy/timeseries.h5'])